### Введение
В рамках отчета был проведен анализ платежных операций с целью выявления fraud-сценариев. 

**Аналитические методы и инструменты:**
* **Фильтрация данных** 
*Фильтрация транзакций по статусу decline/success. 
Фильтрация по валюте (USD, EUR) с целью сохранения релевантности данных*
* **Агрегация данных**
*Агрегация по ключевым признакам
напр.(Customer ID) и дата операции (Operation created at Date.)*
* **Анализ частоты транзакций** 
*Анализ на уровне пользователя и по дате для выявления аномалий. Были выделены пользователи с высоким числом операций за короткий период времени.*
* **Pandas** *Python Библиотека для работы с данными*
* **Jupyter notebook** *Документирование процесса, визуализация*

Для начала, найдем пользователей с рекордным количеством транзакций:

*наприм.
Customer ID 133367 - 458
Customer ID 133367 - 181*


In [130]:
import pandas as pd
file_path = 'antifraud_dataset.xlsx'
df = pd.read_excel(file_path, sheet_name='sheet1')

successful_transactions = df[df['Operation Status'] == 'success']
declined_transactions = df[df['Operation Status'] == 'decline']

# кол-во тразакций/customer id
customers_by_transactions_count = df.groupby('Customer ID').agg(
    transaction_count=('Operation ID', 'count')
).reset_index()

customers_by_transactions_count.sort_values(by='transaction_count', ascending=False).head(10)

,Customer ID,transaction_count
420,133367,458
436,134623,181
1670,27112,169
1677,27308,152
3901,95022,126
1935,32608,108
2005,344501,93
1730,284261,83
471,137952,79
2495,376241,78


### Выявление пользователей с высокой частотой транзакций (Сценарий №1)

Одним из индикаторов fraud-активности является аномально высокая частота транзакций за короткий промежуток времени. Для выявления подобных паттернов был проведён анализ частоты транзакций по каждому пользователю за день. Это могут быть автоматизированные попытки, попадающие в эту категорию могут быть ботами. 

**Наиболее вероятный fraudster**


Customer ID *133367* / Cardholder name *SEG FTJGN* (!) - **284 транзакции за 2022-01-30 и 174 за 2022-01-31**	


*ниже приведены 10 suspect-юзеров*

Для снижения риска таких активностей рекомендуется вводить лимиты на количество операций за определённый период времени и ввести ужесточенные правила проверки транзакций.


In [131]:
transaction_frequency = df.groupby(['Customer ID', 'Operation created at Date']).agg(
    transaction_count=('Operation ID', 'count')
).reset_index()

frequent_transactions = transaction_frequency[transaction_frequency['transaction_count'] > 10]

frequent_transactions.sort_values(by='transaction_count', ascending=False).head(10)

,Customer ID,Operation created at Date,transaction_count
436,133367,2022-01-30,284
437,133367,2022-01-31,174
1732,27112,2022-02-27,169
453,134623,2022-02-27,165
1739,27308,2022-02-26,121
2143,32608,2022-02-26,108
2892,376241,2022-03-24,78
3323,54996,2022-02-26,77
761,16983097,2022-03-30,72
1897,2975035,2022-03-30,72


### Наибольшие общие суммы попыток транзакций за день (Cценарий №2)
Стоит обратить внимание на большое количество отклонённых (по разным причинам) транзакций. Это может свидетельствовать о различных попытках обойти правила платежной системы или проведения подозрительных операций, которые блокируются провайдером или шлюзом.

**Наиболее очевидные fraudster-ы:**

Customer ID *358203* , *Нигерия* (нетипичная страна для совершения тразакций на большую сумму, + взята на контроль ФАТФ), внушительная сумма всех попыток транзакций - *19000 EUR*  за 9 попыток  
    
Customer ID *74457*, *137952*, *27112* . У данных пользователей 57, 46 и 168 отклоненных транзакций на сумму *10338 EUR*, *9467 EUR* и *9373 EUR* соответственно.


Для митигации рисков как и в предыдущем случае рекомендуется внедрение более строгих правил верификации транзакций, автоматическое отслеживание отклонённых операций и предупреждение таких пользователей о необходимости дополнительных проверок.


In [136]:
filtered_df = declined_transactions[declined_transactions['Channel Currency'].isin(['EUR', 'USD'])]

# Группировка по юзеру, дате, валюте 
total_amount_per_day = filtered_df.groupby(['Customer ID', 'Operation created at Date', 'Channel Currency']).agg(
    transaction_count=('Operation ID', 'count'),
    total_amount=('Channel Amount', 'sum')
).reset_index()

total_amount_per_day.sort_values(by='total_amount', ascending=False).head(10)

,Customer ID,Operation created at Date,Channel Currency,transaction_count,total_amount
955,358203,2022-03-01,EUR,9,19000.00
1279,74457,2022-02-26,EUR,57,10338.00
175,137952,2022-02-27,EUR,46,9467.56
391,27112,2022-02-27,EUR,168,9373.42
400,276654,2022-02-22,EUR,6,8100.00
113,124221,2022-02-28,USD,36,6538.90
1065,46536,2022-02-27,EUR,28,6328.00
392,27308,2022-02-26,EUR,116,5394.40
778,351981,2022-02-14,USD,8,4518.40
150,133367,2022-01-30,USD,167,4025.54


### Наибольшие общие суммы успешных транзакций за день 
Аномальных сумм транзакций не выявлено, можно обратить внимание на количество транзакций у **Customer ID *348231*:**

**14 за 2022-02-07, и 12 за 2022-02-06**	

*в следующей таблице (Наибольшее успешное кол-во транзакций за день (в рамках одного пользователя)) приоритизация количества успешных транзакций над их суммой*

In [138]:
filtered_df = successful_transactions[successful_transactions['Channel Currency'].isin(['EUR', 'USD'])]

total_amount_per_day = filtered_df.groupby(['Customer ID', 'Operation created at Date', 'Channel Currency']).agg(
    transaction_count=('Operation ID', 'count'),
    total_amount=('Channel Amount', 'sum')
).reset_index()

total_amount_per_day.sort_values(by='total_amount', ascending=False).head(10)

,Customer ID,Operation created at Date,Channel Currency,transaction_count,total_amount
352,2fa7473923c5e034a231006f12d2a5b4dcd1c920af270f...,2021-12-29,USD,1,1411.76
256,276654,2022-02-22,EUR,1,1350.00
689,356825,2022-02-26,EUR,10,1280.00
532,348231,2022-02-07,EUR,14,1270.95
962,8fd84cacfd03a7678df335b5ca2a5a3abbcf267022c635...,2021-12-29,USD,2,1176.46
531,348231,2022-02-06,EUR,12,1150.00
68,124221,2022-02-27,USD,6,929.57
51,100466,2022-02-26,EUR,4,904.00
951,8cbb61677ead760bdf9420619cf99be667b8fd70010104...,2021-12-29,USD,8,870.53
882,74457,2022-02-26,EUR,4,851.00


## Наибольшее успешное кол-во транзакций за день (в рамках одного пользователя)
За неочевидностью предыдущей таблицы, требуется внимательнее рассмотреть частоту транзакций.

Наиболее подозрительными являются:

**Customer ID 32608, 19 тразакций за 2022-02-26**

Вышеупомянутый **Customer ID 348231, 26 транзакций за 2 дня**

Пользователи, которые совершают большое количество транзакций за день, могут быть вовлечены в card testing или схему быстрого обналичивания средств. Подобные действия направлены на обход ограничений или проверку валидности платёжных данных. Для снижения рисков подобных ситуаций требуются алгоритмы, автоматически блокирующие пользователей с подозрительной активностью.



In [134]:
total_amount_per_day.sort_values(by='transaction_count', ascending=False).head(10)

,Customer ID,Operation created at Date,Channel Currency,transaction_count,total_amount
435,32608,2022-02-26,EUR,19,805.72
532,348231,2022-02-07,EUR,14,1270.95
531,348231,2022-02-06,EUR,12,1150.00
984,95022,2022-02-26,EUR,12,652.00
458,343271,2022-02-04,EUR,12,465.00
89,134623,2022-02-27,EUR,11,241.41
985,95022,2022-02-27,EUR,11,519.00
689,356825,2022-02-26,EUR,10,1280.00
268,279660,2022-02-27,EUR,9,195.00
369,308454,2022-02-13,EUR,8,199.00


### Наибольшее кол-во отклоненных транзакций у пользователя
 Ниже указаны пользователи с наибольшим количеством отклонённых транзакций. В предоставленной базе таких юзеров гораздо больше, первые 10 указаны для наглядности.

*прим.* Customer ID 133367 / Cardholder name SEG FTJGN (!) - 284 транзакции за 2022-01-30 и 171 за 2022-01-31. Пользователь с невнятным именем уже был отмечен за частоту транзакций. Из 458 попыток за 2 дня, 3 оказались успешными.

Все нижеприведенные пользователи с огромной вероятностью являются  fraudster-ами.





In [139]:
total_declined_per_day = declined_transactions.groupby(['Customer ID', 'Operation created at Date', 'Operation Status']).agg(
    transaction_count=('Operation ID', 'count'),
).reset_index()

total_declined_per_day.sort_values(by='transaction_count', ascending=False).head(10)

,Customer ID,Operation created at Date,Operation Status,transaction_count
275,133367,2022-01-30,decline,284
276,133367,2022-01-31,decline,171
1083,27112,2022-02-27,decline,168
284,134623,2022-02-27,decline,154
1085,27308,2022-02-26,decline,116
1291,32608,2022-02-26,decline,89
1794,376241,2022-03-24,decline,78
2006,54996,2022-02-26,decline,75
1156,2975035,2022-03-30,decline,72
492,16983097,2022-03-30,decline,72
